## Install the matrice package or upgrade it if already installed

In [ ]:
!python -m pip install --upgrade matrice

## Setting up the credentials
*   ENV
*   MATRICE_ACCESS_KEY_ID
*   MATRICE_SECRET_ACCESS_KEY
*   MATRICE_ACCOUNT_NUMBER

In [ ]:
import yaml
import os

class Vars:
    _data = {}
    _path = None

    @classmethod
    def initialize(cls, path):
        cls._path = path
        cls._data = cls.load()

    @classmethod
    def load(cls):
        if cls._path and os.path.exists(cls._path):
            with open(cls._path, 'r') as f:
                return yaml.safe_load(f)
        return {}

    @classmethod
    def get(cls, key):
        if key not in cls._data:
            raise KeyError(f"Variable {key} not found")
        return cls._data[key]

    @classmethod
    def set(cls, key, value):
        cls._data[key] = value
        with open(cls._path, 'w') as f:
            yaml.safe_dump(cls._data, f, default_flow_style=False)


In [ ]:
Vars.initialize('vars.yaml')

In [ ]:
Vars.set("ENV", "prod")
Vars.set("MATRICE_ACCESS_KEY_ID", "enter access key")
Vars.set("MATRICE_SECRET_ACCESS_KEY", "enter secret access key")
Vars.set("MATRICE_ACCOUNT_NUMBER", "enter your matrice acc number")

In [ ]:
import os
os.environ['ENV'] = Vars.get('ENV')
access_key = Vars.get('MATRICE_ACCESS_KEY_ID')
secret_key = Vars.get('MATRICE_SECRET_ACCESS_KEY')
account_number = Vars.get('MATRICE_ACCOUNT_NUMBER')

## Import required matrice modules

In [ ]:
from matrice.session import Session
from matrice.projects import Projects
from matrice.dataset import Dataset
from matrice.models import Model
from matrice.deployment import Deployment
from matrice.exported_model import ExportedModel
import sys
import os

## Create Object Detection Project with Matrice SDK

1.   Initialize matrice connection session
2.   List projects in user account
3.   Create the Project
4.   Update the session with the project_id









In [ ]:
session_instance = Session(account_number=account_number, access_key=access_key, secret_key=secret_key)
print("A Session has been initialized:", session_instance)

A Session has been initialized: <matrice.session.Session object at 0x00000193E3DB2560>


In [ ]:
projects_list, msg = session_instance.list_projects(project_type="instance_segmentation")

# Check if there is a message or error
if msg:
    print(f"Message: {msg}")

# Print the projects in a formatted manner
for project_name, project_instance in projects_list.items():
    print(f"Project Name: {project_name} | Project ID: {project_instance.project_id}") #To fix: ERROR: May return none and give error if proj is empty list

Message: Projects fetched successfully
Project Name: field_water_segmentation | Project ID: 6826f0240a1d0e31e05fce17
Project Name: Brain_tumor_sef_final | Project ID: 6825a95f0a1d0e31e05fc731
Project Name: road_view_seg | Project ID: 6826e9b90a1d0e31e05fcdcf
Project Name: kidney_tumor | Project ID: 6825ba920a1d0e31e05fc7a6
Project Name: brain tumor segmentation | Project ID: 682556ce0a1d0e31e05fc59d
Project Name: organ_segmentation | Project ID: 6823435d0a1d0e31e05fbee9
Project Name: final_theft | Project ID: 68231ab80a1d0e31e05fbded
Project Name: Shoplifting_testing_1 | Project ID: 6822d72d0a1d0e31e05fbbc2
Project Name: prod_def_2 | Project ID: 6821e71c0a1d0e31e05fb848
Project Name: test_round-1 | Project ID: 682132ef0a1d0e31e05fb36f


In [ ]:
project_name = "Brain_tumor_sef_final"
for name, project_instance in projects_list.items():
    if name == project_name:
        projects_instance = Projects(session_instance, project_name=project_name)
        session_instance.update(projects_instance.project_id)
        print(f"Project '{project_name}' already exists. Initialized Projects instance.")
        project_exists = True
        break
    else:
        project_exists = False

if project_exists == False:
    print(f"Creating a new project: {project_name}")

Project 'Brain_tumor_sef_final' already exists. Initialized Projects instance.


In [ ]:
import pprint

name = project_name

# Function to create a project
def create_project():
    project_name = name
    input_type = "image"
    output_type = "instance_segmentation"

    enabled_platforms = {
        "matrice": True,
        "android": False,
        "ios": False,
        "tpu": False,
        "intelCPU": False,
        "gcloudGPU": False
    }

    resp, error = session_instance._create_project(project_name, input_type, output_type)
    if error:
        print(f"Error: {error}")
        return None, None
    else:
        print(f"Project created with ID: {resp['_id']}")
        return resp['_id'], resp['name']

# Check if project_id and project_name exist
project_id = None
project_name = None

if not project_exists:
    project_id, project_name = create_project()
else:
    project_id = projects_instance.project_id
    project_name = projects_instance.project_name
    print(f"Project already exists with ID: {project_id} and Name: {project_name}")


print("----------------------------")
print(f"Project ID: {project_id}")
print(f"Project Name: {project_name}")
print("----------------------------")


Project already exists with ID: 6825a95f0a1d0e31e05fc731 and Name: Brain_tumor_sef_final
----------------------------
Project ID: 6825a95f0a1d0e31e05fc731
Project Name: Brain_tumor_sef_final
----------------------------


In [ ]:
Vars.set("project_name",project_name)
Vars.set("project_id", project_id)

In [ ]:
session_instance.update(project_id)
project_instance = Projects(session_instance, project_name=project_name)
print("A Project class has been initialized",project_instance)

# From next time, for deployment run till here

A Project class has been initialized <matrice.projects.Projects object at 0x00000193E30C85E0>


## Import the Dataset to the project
1. Upload the dataset
2. Start a data import action
3. Check the status of the dataset import action
4. Check the processed dataset information
5. Upload the dataset

For uploading the dataset you have 2 ways -

* Upload the dataset to cloud storage bucket with project.upload_cloud_dataset()
* Upload the dataset from your local storage with project.upload_local_dataset()

In [ ]:
dataset = project_instance.import_local_dataset(
    project_type='instance_segmentation',
    dataset_name='Brain_tumor_final_dataset',
    file_path = r"D:\d\brain tumor.zip",
    dataset_type = "instance_segmentation",
    input_type = "mscoco"
) #THIS runs on backend. CHECK UI and then fetch ds id manually from UI

In [ ]:
dataset=Dataset(session_instance,dataset_id='6825a98977c12201804e5b8c') #To EDIT

In [ ]:
dataset_id = dataset.dataset_id
# action_id = dataset_action.action_id

print(f"Dataset ID: {dataset_id}")
# print(f"Action ID: {action_id}")

Dataset ID: 6825a98977c12201804e5b8c


In [ ]:
Vars.set("dataset_id", dataset.dataset_id)

In [ ]:
def print_dataset_info(dataset):

    dataset_id = dataset.dataset_details['_id']
    dataset_name = dataset.dataset_details['name']
    version_status = dataset.dataset_details.get("stats", [{}])[0].get("versionStatus")
    latest_version = dataset.dataset_details['latestVersion']
    no_of_samples = sum(version['versionStats']['total'] for version in dataset.dataset_details.get('stats', []))
    no_of_classes = len(dataset.dataset_details.get('stats', [{}])[0].get('classStat', {}))
    no_of_versions = len(dataset.dataset_details.get('allVersions', []))
    last_updated_at = dataset.dataset_details.get('updatedAt')

    print(f"Dataset ID: {dataset_id}")
    print(f"Dataset Name: {dataset_name}")
    print(f"Version Status: {version_status}")
    print(f"Latest Version: {latest_version}")
    print(f"Number of Samples: {no_of_samples}")
    print(f"Number of Classes: {no_of_classes}")
    print(f"Number of Versions: {no_of_versions}")
    print(f"Last Updated At: {last_updated_at}")

dataset.refresh()
print_dataset_info(dataset)

Dataset ID: 6825a98977c12201804e5b8c
Dataset Name: Brain_tumor_final_dataset
Version Status: processed
Latest Version: v1.0
Number of Samples: 3203
Number of Classes: 1
Number of Versions: 1
Last Updated At: 2025-05-15T08:48:09.981Z


##Model Family and Architecture

In [ ]:
from matrice.model_store import get_all_model_families

resp , error , mesage = get_all_model_families( session_instance , project_id , project_type="instance_segmentation" )
print(error)
print("----List of available Detection models on platform----")
# Iterate through each model family in the response
for model_family in resp:
    # Extract _id and modelFamily (model family name)
    model_family_id = model_family.get('_id')
    name = model_family.get('modelFamily')

    # Print in formatted manner
    print(f"ID: {model_family_id}, Model Family Name: {name}")

None
----List of available Detection models on platform----
ID: 67625ff9d6db1ef9132e84e2, Model Family Name: YOLOv8_Instance_Segmentation
ID: 6762c9b45892e63f791b755d, Model Family Name: YOLOv9_Instance_Segmentation


In [ ]:
from matrice.model_store import ModelFamily

model_family_id = "67625ff9d6db1ef9132e84e2" #To EDIT from previous result
# Initialize the ModelFamily instance after choosing a Model Family
model_family_instance = ModelFamily(session_instance, model_family_id=model_family_id) # ResNet
print("A ModelFamily instance has been initialized",model_family_instance)

A ModelFamily instance has been initialized <matrice.model_store.ModelFamily object at 0x00000193E3E6B9D0>


In [ ]:
# Fetch model architectures
import pprint
arch_resp, error, message = model_family_instance.get_model_archs()
if error:
    print(f"Error: {error}")
else:
    pprint.pprint(arch_resp)
    # Check if resp is a list or a dictionary
    if isinstance(arch_resp, list):
        # Iterate through each model info in the list
        for model_info in arch_resp:
            model_key = model_info["model_key"]
            model_arch_instance = model_info["model_arch_instance"]

            # Extract model information
            model_info_id = model_arch_instance.model_arch_id
            name = model_arch_instance.model_name
            key = model_arch_instance.model_key
            params = model_arch_instance.params_millions

            # Print in formatted manner
            print(f"ID: {model_info_id} |  Model Name: {name} | Model Key: {key} | Params in Millions: {params}")

    elif isinstance(arch_resp, dict):
        # Iterate through each model key in the dictionary
        for model_key, model_arch_instance in arch_resp.items():
            # Extract model information
            model_info_id = model_arch_instance.model_arch_id
            name = model_arch_instance.model_name
            key = model_arch_instance.model_key
            params = model_arch_instance.params_millions

            # Print in formatted manner
            print(f"ID: {model_info_id} | Model Name: {name} | Model Key: {key} | Params in Millions: {params}")

{'yolov8l_seg': <matrice.model_store.ModelArch object at 0x00000193E4309FC0>,
 'yolov8m_seg': <matrice.model_store.ModelArch object at 0x00000193E430AFE0>,
 'yolov8n_seg': <matrice.model_store.ModelArch object at 0x00000193E4310160>,
 'yolov8s_seg': <matrice.model_store.ModelArch object at 0x00000193E400C700>,
 'yolov8x_seg': <matrice.model_store.ModelArch object at 0x00000193E4309090>}
ID: 67625ff9d6db1ef9132e84e3 | Model Name: YOLOv8 Instance Segmentation Nano | Model Key: yolov8n_seg | Params in Millions: 3.4
ID: 67625ff9d6db1ef9132e84e4 | Model Name: YOLOv8 Instance Segmentation Small | Model Key: yolov8s_seg | Params in Millions: 11.8
ID: 67625ff9d6db1ef9132e84e5 | Model Name: YOLOv8 Instance Segmentation Medium | Model Key: yolov8m_seg | Params in Millions: 27.3
ID: 67625ff9d6db1ef9132e84e6 | Model Name: YOLOv8 Instance Segmentation Large | Model Key: yolov8l_seg | Params in Millions: 46
ID: 67625ff9d6db1ef9132e84e7 | Model Name: YOLOv8 Instance Segmentation Xtra Large | Model Ke

In [ ]:
from matrice.model_store import ModelArch
train_arch = 'yolov8s_seg' #To EDIT from previous result
print("Chosen Training Architecture :" ,train_arch)
model_arch_instance = arch_resp.get(train_arch)
print("Model Architecture instance initialized for chosen training architecture :", model_arch_instance)

In [ ]:
import pprint
train_config = model_arch_instance.get_train_config()
print("-----Default Train Config------")
pprint.pprint(train_config)
print("--------------------------------")
print("-------Training instance initialized--------")
# print(train_instance)

##configuring model's hyperparameter

In [ ]:
model_config = train_config['model_config']

# Modify the model_config as needed
# modifications:
#model_config['batch_size'] = [8]
model_config['epochs'] = [90]  #These are hyper-parameters and can be changed
model_config['learning_rate']=[0.01]


# Repass the modified model_config into the train_config
train_config['model_config'] = model_config

# Print the updated train_config
print("-----Updated Model Config------")
pprint.pprint(train_config['model_config'])
print("--------------------------------")

# Print the updated train_config
print("-----Updated Train Config------")
pprint.pprint(train_config)
print("--------------------------------")

In [ ]:
train_resp , msg , err = project_instance.add_models_for_training(train_config,primary_metric='mAP@50',dataset_id=dataset.dataset_id,dataset_name=dataset.dataset_name,)
print("------------Model added for training----------")
pprint.pprint(train_resp)

In [ ]:
import time
from matrice.action import Action

# Define the action ID and the status you want to check
action_id = train_resp[0]['_idActionStatus']
success_status = 'SUCCESS'
time_sec = 90

# Function to check the status of the action
def check_action_status(S, action_id):
    action_instance = Action(S, action_id)
    print("---------------------")
    print(f"Action ID: {action_id}")
    print(f"Current Status: {action_instance.status}")
    print(f"Step Code: {action_instance.step_code}")
    print(f"Action service is : {action_instance.service_name}")
    print("---------------------")
    return action_instance.status

# Loop to check status every 2 minutes until it is 'success'
while True:
    status = check_action_status(session_instance, action_id)
    if status == success_status:
        print("Action status is 'success'. Model is successfully trained.")
        break
    else:
        print(f"Model is training. Waiting {time_sec} seconds before checking again.")
        time.sleep(time_sec)


##analyizing results


In [ ]:
model_id = train_resp[0]['_id']
print(f"Model ID from response data: {model_id}")


# Initialize the Model class with the model_id
model_instance = Model(session_instance, model_id)
print("A Model instance has been initialized : ", model_instance)

In [ ]:
# Print the test score, validation score, and best epoch
print(f"Test Score: {model_instance.test_score}")
print(f"Validation Score: {model_instance.val_score}")
print(f"Best Epoch: {model_instance.best_epoch}")

In [ ]:
Vars.set("model_train_id", model_instance.model_id)

##plotting losses and metrics per epoch


In [ ]:
model_instance.plot_epochs_losses()

In [ ]:
model_instance.plot_epochs_metrics()

In [ ]:
model_instance.plot_eval_results()

##Deployment

In [ ]:
dep_name="deployment_bt"
deployment = project_instance.create_fastapi_deployment(
    model_id = "6825b955ba25d9750f40e262",  # It can also be model_train_id depending on the model you want to deploy
    deployment_name = dep_name,
    shutdown_threshold=25
) #again this WILL GIVE ERROR BUT IT WORKS ON BACKEND. CHECK UI

In [ ]:
deployment=Deployment(session_instance,deployment_id='68271752bc5142b714584535', deployment_name="deployment_bt") #fetch dp id manually from UI and name can be changed

In [ ]:
key_resp = deployment.create_auth_key(1) # This aut key is valid for 1 day, used by client to access models, dataset, etc.
# auth_key = key_resp['key']
auth_key = key_resp[0]['key']
print(auth_key)

In [ ]:
Vars.set("deployment_id", deployment.deployment_id)
Vars.set("deployment_name", deployment.deployment_name)
Vars.set("auth_key", auth_key)

##prediction

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import glob
import os
import numpy as np
import random
import json  # For pretty-printing

# Images path
image_paths = glob.glob("enter  adress to your test image here")

# Helper for random transparent color
def random_color():
    return tuple(random.choices(range(256), k=3)) + (150,)

# Load font with fallback
def get_font(font_size=40):
    try:
        return ImageFont.truetype("arial.ttf", font_size)
    except:
        return ImageFont.load_default()

for photo in image_paths:
    print(f"\nProcessing: {photo}")
    result = deployment.get_prediction(photo, auth_key)

    predictions = result
    img = Image.open(photo).convert("RGBA")
    overlay = Image.new("RGBA", img.size, (255, 255, 255, 0))
    draw = ImageDraw.Draw(overlay)
    font = get_font(max(16, img.width // 40))

    found_mask = False

    for pred in predictions:
        category = pred.get("category")
        confidence = pred.get("confidence")
        masks = pred.get("masks", [])

        if masks and isinstance(masks[0], list) and len(masks[0]) >= 6:
            found_mask = True
            polygon_coords = masks[0]  # Use first mask
            polygon = [(int(polygon_coords[i]), int(polygon_coords[i+1])) for i in range(0, len(polygon_coords), 2)]
            color = random_color()
            draw.polygon(polygon, fill=color)

            x1, y1 = polygon[0]
            label = f"{category} {confidence:.2f}"

            text_size = draw.textbbox((x1, y1), label, font=font)
            text_w = text_size[2] - text_size[0]
            text_h = text_size[3] - text_size[1]

            padding = 4
            bg_box = [x1 - padding, y1 - text_h - padding, x1 + text_w + padding, y1 + padding]
            draw.rectangle(bg_box, fill=(0, 0, 0, 255))
            draw.text((x1, y1 - text_h), label, font=font, fill=(255, 255, 255, 255))

    if not found_mask:
        print("No usable segmentation masks found in predictions.")

    combined = Image.alpha_composite(img, overlay)

    display_size = (int(combined.width * 1.5), int(combined.height * 1.5))
    display_img = combined.resize(display_size, Image.BICUBIC)

    plt.figure(figsize=(12, 12))
    plt.imshow(display_img)
    plt.axis('off')
    plt.title(os.path.basename(photo))
    plt.show()
